# Интеллектуальный анализ данных – весна 2024
# Домашнее задание 6: классификация текстов

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [344]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [345]:
!wget https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw06_texts/tweets_coronavirus.csv

--2024-05-13 20:26:05--  https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw06_texts/tweets_coronavirus.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8829280 (8.4M) [text/plain]
Saving to: ‘tweets_coronavirus.csv.5’

tweets_coronavirus. 100%[===================>]   8.42M  --.-KB/s    in 0.09s   

2024-05-13 20:26:05 (94.6 MB/s) - ‘tweets_coronavirus.csv.5’ saved [8829280/8829280]



In [346]:
df = pd.read_csv('/content/tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
1380,5469,50421,Dublin City Centre,17-03-2020,Give the perfect gift this Mother's Day from o...,Extremely Positive
16567,23971,68923,"ÃÂT: 36.113451,-115.173098",24-03-2020,2/ There are so many (-) indicators that shoul...,Negative
11089,17257,62209,"Minnesota, USA",21-03-2020,Millions of Americans on social security or st...,Negative
29451,39952,84904,NaN,09-04-2020,A big thank you NHS workers all over the count...,Extremely Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная эмоциональная окраска и 0 - если отрицательная.

In [347]:
d={'Extremely Positive':1,'Positive':1,'Negative':0,'Extremely Negative':0} #кодирую 1- позитив, 0 - негатив
df['Sentiment']=df['Sentiment'].map(d) #само преобразование

In [348]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


Сбалансированы ли классы?

In [349]:
print(df['Sentiment'].value_counts())

Sentiment
1    18046
0    15398
Name: count, dtype: int64


**Ответ:** несмотря на разницу в 3 тысячи, в целом, классы сбалансированы, с данными работать можно.

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [350]:
df.isnull().sum() #Пропуски в локации

UserName            0
ScreenName          0
Location         7049
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [351]:
df['Location']=df['Location'].fillna('Unknown')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,Unknown,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,Unknown,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


In [352]:
df.isnull().sum() #Пропусков в локации больше нет

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

Разделите данные на обучающие и тестовые в соотношении 7 : 3 и `random_state=0`

In [353]:
y = df['Sentiment']
X= df.drop('Sentiment', axis=1) #дроп целевой переменной из выборки

In [354]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=0) #заранее делю на таргет, мне так удобнее работать с данными

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведение токенов в нижний регистр

In [355]:
text=X_train['OriginalTweet']
d={}
for i in text:
  spl=i.lower().split()
  for t in spl:
    if t not in d:
      d[t]=1
    else:
      d[t]+=1

Какой размер словаря получился?

In [356]:
print("Размер словаря:", len(d))

Размер словаря: 79755


Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе

In [357]:
t=sorted(d.items(),key=lambda x:x[1],reverse=True)[:10]
for i in t:
  print (i[0],i[1])

the 26815
to 23373
and 14684
of 13012
a 11737
in 11198
for 8566
#coronavirus 8223
is 7383
are 7050


**Ответ:** Такие токены в топе, так как это самые частоупотербимые языковые конструкции.

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [358]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #загружаю стопслова из библ
sw = {t: n for t, n in d.items() if t not in stopwords.words('english')} #использую генератор словаря с условием о англ стопсловах
t=sorted(sw.items(),key=lambda x:x[1],reverse=True)[:10] #сортирую по возр, отбираю самые большие знач
for i in t:
  print (i[0],i[1])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#coronavirus 8223
prices 3891
food 3820
grocery 3469
supermarket 3288
people 3175
covid-19 3173
store 3155
#covid19 2471
&amp; 2314


**Ответ:**  Больше всего в твиттах затрагивается тема коронавируса, что демонстрируется хэштегом. Людей волнуют цены, скорее всего на еду, так как слова по частотности идут рядом (еда, продольствие, супермаркеты).Из списка пропали стоп-слова - неинформативные языковые конструкции, которые используются из-за особенностей грамматики

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [359]:
t=sorted(d.items(),key=lambda x:x[1])[:20]
for i in t:
  print (i[0],i[1])

https://t.co/1m881cwfuv 1
happy.. 1
https://t.co/z0intks34x 1
mnuchinãâs 1
brink. 1
https://t.co/jmobv8z0u0 1
university's 1
teaching.) 1
@catholicpres 1
@catholicuniv 1
https://t.co/evqby035wf 1
https://t.co/riqrhxxeim 1
@@ballardspahrll 1
#aca 1
easier...take 1
ãânecessaryãâ 1
https://t.co/0fmsmlgepm 1
husted: 1
irishman 1
#happystpatricksday! 1


**Ответ:** Токены непопулярны, так как в рамках темы используются реже всего (#деньсвятогопатрика, например, мало подходит к дискурсу о короне). Все остальное либо уникальные сслыки, либо отметки, либо просто очень непопулярные слова (сложные формы слов).


Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [360]:
from nltk.tokenize import TweetTokenizer
tw=TweetTokenizer()
d={}
for i in X_train['OriginalTweet']:
  i=i.lower()
  tok=tw.tokenize(i)
  for w in tok:
    if w not in d:
      d[w]=1
    else:
      d[w]+=1
t=sorted(d.items(),key=lambda x:x[1],reverse=True)[:10]
for i in t:
  print (i[0],i[1])

the 26993
. 24118
to 23478
, 17571
and 14825
of 13044
a 11891
in 11348
? 9524
#coronavirus 8808


**Ответ:** Появились знаки препинания. Твитт токенайзер выделяет их, так как они могут быть значимы (скобочка, например, может сигнализировать о положительной тональности твитта).

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [361]:
from string import punctuation
sw = {t: n for t, n in d.items() if (t not in stopwords.words('english') and t not in punctuation)}
t=sorted(sw.items(),key=lambda x:x[1],reverse=True)[:10]
for i in t:
  print (i[0],i[1])

#coronavirus 8808
â 7415
 7311
19 7167
covid 6253
prices 4601
 4372
food 4367
store 3877
supermarket 3805


**Ответ:** Слова есть, но все остальное вызывает вопросы. Это просто какие-то невнятные знаки (или артикль даже).


Удалите из словаря токены из одного символа, с позицией в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [362]:
sw = {t: n for t, n in d.items() if (t not in stopwords.words('english') and t not in punctuation)
and not (len(t)==1 and (ord(t)>=128))}
t=sorted(sw.items(),key=lambda x:x[1],reverse=True)[:10]
for i in t:
  print (i[0],i[1])
print('\n')
m=sorted(sw.items(),key=lambda x:x[1])[:20]
for i in m:
  print (i[0],i[1])

#coronavirus 8808
19 7167
covid 6253
prices 4601
food 4367
store 3877
supermarket 3805
grocery 3523
people 3463
#covid19 2589


https://t.co/1m881cwfuv 1
https://t.co/z0intks34x 1
mnuchinã 1
https://t.co/jmobv8z0u0 1
university's 1
@catholicpres 1
@catholicuniv 1
https://t.co/evqby035wf 1
https://t.co/riqrhxxeim 1
@ballardspahrll 1
#aca 1
https://t.co/0fmsmlgepm 1
irishman 1
#happystpatricksday 1
guiness 1
https://t.co/18v0pyhwb7 1
https://t.co/ls0g86i8pu 1
https://t.co/ofvkejff23 1
https://t.co/xoodgd2ulx 1
lifestyles 1


**Ответ:** Топ популярных стал бесполезнее - в него вошли неинформативные числа, например. Топ непопулярных стал полезнее - больше ссылок и отметок людей, меньше слов не по теме

Выведите топ-10 популярных хештегов с количеством встреч. Что можно сказать о них?

In [363]:
sw = {t: n for t, n in d.items() if t[0]=='#'}
t=sorted(sw.items(),key=lambda x:x[1],reverse=True)[:10]
for i in t:
  print (i[0],i[1])

#coronavirus 8808
#covid19 2589
#covid_19 1734
#covid2019 946
#toiletpaper 744
#covid 641
#socialdistancing 465
#coronacrisis 448
#pandemic 257
#coronaviruspandemic 249


**Ответ:** Люди ставили на посты общий хэштег про коронавирус, вероятно, для поднятия виральности. Людей волновала ситуация с туалетной бумагой, с социальной дистанцией и кризисом.

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [364]:
sw = {t: n for t, n in d.items() if 'https://t.co/' in t[:13]}
t=sorted(sw.items(),key=lambda x:x[1],reverse=True)[:10]
print(t)

[('https://t.co/oxa7swtond', 5), ('https://t.co/gp3eusapl8', 4), ('https://t.co/deftrui1pfã\x82â', 3), ('https://t.co/wrlhyzizaa', 3), ('https://t.co/kuwipf1kqw', 3), ('https://t.co/zjnrx6dkkn', 3), ('https://t.co/3gbbdpdjat', 3), ('https://t.co/e2znxajpre', 3), ('https://t.co/catkegayoy', 3), ('https://t.co/g63rp042ho', 3)]


## **Ответ:** # Популярность ссылок с популярностью хэштегов не сравнима. Ссылок меньше в разы - максимум 5, хэштегов - больше 8 тысяч одинаковых.

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для изначального выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа, с позицией в таблице Unicode 128 и более и ссылки на t.co



In [365]:
tw=TweetTokenizer()
def custom_tokenizer(text):
  tokens={}
  text=text.lower()
  tok=tw.tokenize(text)
  for t in tok:
    if ('https://t.co/' not in t[:13]) and (t not in stopwords.words('english')) and (t not in punctuation) and not (len(t)==1 and ord(t)>=128):
        #добавить слеш!
        #ord() expected a character, but string of length 6 found
        if t in tokens.items():
          tokens[t]+=1
        else:
          tokens[t]=1
  return tokens


In [366]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

{'sample': 1, 'text': 1, '@sample_text': 1, '#sampletext': 1}

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [367]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=custom_tokenizer)
cvv=cv.fit_transform(X_train['OriginalTweet'])
print(len(cv.vocabulary_))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


45308


**Ответ:** Словарь сильно меньше и к тому же информативнее. В начале словарь был около 75к, сейчас 45к.

Посмотрим на какой-нибудь конкретный твитт:

In [368]:
ind = 9023
X_train.iloc[ind]['OriginalTweet'], y_train.iloc[ind]

('Nice one @SkyNews lets not panic but show ppl in france queueing for food!!! #CoronavirusOutbreak #COVID2019 brainless!! Ffs',
 0)

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [382]:
cv.fit(X_train['OriginalTweet'])
vec=cv.fit_transform([X_train.iloc[ind]['OriginalTweet']])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [383]:
imprt=vec.argmax() #самый важный твитт из вектора
#ЧЕРЕЗ ПРОСТОЙ АРГМИН ВЫДАВАЛСЯ САМЫЙ НЕВАЖНЫЙ ПО ВСЕМУ ДАТАСЕТУ
non_imprt=np.nonzero(vec)[1] #ненулевые токены, их индексы
tok=np.argmin(vec[0,non_imprt]) #самый неважный токен в векторе
print(cv.get_feature_names_out()[imprt])
print(cv.get_feature_names_out()[non_imprt[tok]])

#coronavirusoutbreak
nice


**Ответ:** Честно говоря, данные не очень информативны. С одной стороны, самый популярный токен вроде и полезен - говорит о пандемии, с другой стороны, не очень ясно, почему именно nice оценилось как бесполезное (ведь оно может сигнализировать о положительной тональности твитта)

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [384]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf= TfidfVectorizer(tokenizer=custom_tokenizer)
tf.fit(X_train['OriginalTweet'])
tweet=tf.fit_transform([X_train.iloc[ind]['OriginalTweet']])

In [385]:
imprt=tweet.argmax()
non_imprt=np.nonzero(tweet)[1]
tok=np.argmin(tweet[0,non_imprt])
print(cv.get_feature_names_out()[imprt])
print(cv.get_feature_names_out()[non_imprt[tok]])

#coronavirusoutbreak
ffs


Тут ситуация, на мой взгляд, чуть более оптимистичная. ffs представляет собой просто неясный набор букв и не может быть содержательно интерпрет алгоритмами. Самый важный токен выделен корректно

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [373]:
X_train['Num'] = X_train.reset_index(drop=True).index #создание новых индексов, потому что по старым не выдавались твитты
X_train[X_train['OriginalTweet'].apply(lambda x: 'amazing' in x) & (y_train == 1)]


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Num
4583,9362,54314,"Moulton, England",19-03-2020,Hearing so many stories of NHS heroes Teachers...,411
8221,13787,58739,Unknown,20-03-2020,Let s just take a minute to say THANK YOU also...,888
3183,7654,52606,"London, England",18-03-2020,"Back at the ""Frontline""\r\r\nA massive shout o...",1235
24347,33577,78529,Unknown,05-04-2020,Massive thanks to @waitrose for my delivery of...,1279
3281,7772,52724,"Nairobi, Kenya",18-03-2020,Crisp clean fresh air perfect ambience Covid 1...,1281
8851,14552,59504,Unknown,20-03-2020,With everything going on with 19 shopping onli...,1428
21299,29778,74730,"Morrisville, NC",01-04-2020,We've got newly updated info about online shop...,1753
11808,18128,63080,"Brighton, England",21-03-2020,#CoronaCrisis #cornavirusuk #HelpEachOther can...,1780
11674,17953,62905,Essex,21-03-2020,Thank you so much @waitrose you have been amaz...,2313
10468,16499,61451,Unknown,21-03-2020,This Covid-19 is really an amazing ÃÂselecti...,2803


In [386]:
tweet=tf.fit_transform([X_train.iloc[888]['OriginalTweet']])
pd.set_option('display.max_columns', None)
pd.DataFrame(tweet.toarray(), columns=tf.get_feature_names_out())
#странность с кодом, но раньше выводилось спасибо как 2Х упоминание. Его вес был около 0.33

,19,also,amazing,behind,cards,click,delivered,getting,let,many,minute,online,people,postal,real,say,shopping,take,thank,workers
0,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607,0.223607


**Ответ:** Больший вес имеет слово "Спасибо". Оно используется несколько раз в векторе и помогает понять, что окраска у твитта положительная.

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [375]:
tf=TfidfVectorizer(tokenizer=custom_tokenizer)
cv=CountVectorizer(tokenizer=custom_tokenizer)
tf_tr=tf.fit_transform(X_train['OriginalTweet'])
tf_te=tf.transform(X_test['OriginalTweet'])
cv_tr=cv.fit_transform(X_train['OriginalTweet'])
cv_te=cv.transform(X_test['OriginalTweet'])
#целевая переменная была выделена с самого начала, потому что так было удобнее работать с данными

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

In [376]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr=LogisticRegression(max_iter=500)
lr.fit(cv_tr,y_train)
cvv_tr=lr.predict(cv_tr)
cvv_te=lr.predict(cv_te)
lr.fit(tf_tr,y_train)
tff_tr=lr.predict(tf_tr)
tff_te=lr.predict(tf_te)
cv_as_tr=accuracy_score(cvv_tr,y_train)
cv_as_te=accuracy_score(cvv_te,y_test)
tf_as_tr=accuracy_score(tff_tr,y_train)
tf_as_te=accuracy_score(tff_te,y_test)
print('CV Accuracy Score train:',cv_as_tr)
print('CV Accuracy Score test:',cv_as_te)
print('TFiDF Accuracy Score train:',tf_as_tr)
print('TFiDF Accuracy Score test:',tf_as_te)

KeyboardInterrupt: 

**Ответ: Accuracy высокая и на трейне, и на тесте, но в случае с CV все же выше.

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
from nltk.stem.snowball import SnowballStemmer
st= SnowballStemmer(language='english')
def custom_stem_tokenizer(text):
  tokens={}
  text=text.lower()
  tok=tw.tokenize(text)
  for t in tok:
    if ('https://t.co/' not in t[:13]) and (t not in stopwords.words('english')) and (t not in punctuation) and not (len(t)==1 and ord(t)>=128):
        t=st.stem(t)
        if t in tokens.items():
          tokens[t]+=1
        else:
          tokens[t]=1
  return tokens

In [ ]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

In [ ]:
cv = CountVectorizer(tokenizer=custom_stem_tokenizer)
cv_snow=cv.fit_transform(X_train['OriginalTweet'])
tf=TfidfVectorizer(tokenizer=custom_stem_tokenizer)
tf_snow=tf.fit_transform(X_train['OriginalTweet'])
print(len(cv_snow.vocabulary_))
print(len(tf_snow.vocabulary_))

**Ответ** Размер словаря стал меньше. Стемминг отбрасывает аффиксы, сохраняя только начальную форму слова, поэтому это закономерно.

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
cv_snow_te=cv.transform(X_test['OriginalTweet'])
tf_snow_te=tf.transform(X_test['OriginalTweet'])
lr.fit(cv_snow,y_train)
cv_snow_tr=lr.predict(cv_snow)
cv_snow_te=lr.predict(cv_snow_te)
lr.fit(tf_snow,y_train)
tf_snow_tr=lr.predict(tf_snow)
tf_snow_te=lr.predict(tf_snow_te)
print('CV Accuracy Score train snow:',cv_snow_tr)
print('CV Accuracy Score test snow:',cv_snow_te)
print('TFiDF Accuracy Score train snow:',tf_snow_tr)
print('TFiDF Accuracy Score test snow:',tf_snow_te)

**Ответ:** Качество поменялось незначительно. Вероятно, произошло это из-за того, что в постах используются в целом одинаковые формы слова, когда обсуждается какая-то тема. В конкретном случае стемминг использовать излишне

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=0.3
                        ).fit(X_train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

**Ответ:** Возможно это связано с тем, что самые частые слова могут быть не очень полезными (в данном датасете это общие хэштеги), поэтому потолок такой небольшой


Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=11
                        ).fit(X_train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

In [ ]:
cv_tr=cv_df.transform(X_train['OriginalTweet'])
cv_te=cv_df.transform(X_test['OriginalTweet'])
lr= LogisticRegression(max_iter=500)
lr.fit(cv_tr, y_train)
cv_df=lr.predict(cv_tr)
accuracy = accuracy_score(y_test, cv_df)
print(accuracy)


**Ответ:** Размер словаря 3700. С аккюраси вышла катастрофа, и исправить я ее не успела


В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `berplot` содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
scl=StandardScaler(with_mean=False)
#не хотело центрироваться со средними
# Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.
lr=LogisticRegression(max_iter=500)
vec=cv_df.fit_transform(X_train['OriginalTweet'])

In [ ]:
data=scl.fit_transform(vec)
lr.fit(data,y_train)
c=lr.coef_[0]
pol=np.argsort(c)[:-10]
otr=np.argsort(c)[:10]
name=cv_df.get_feature_names_out()
print([name[i] for i in pol])
print([name[i] for i in otr])

In [ ]:
plt.figure(figsize=(9,3))
plt.barh([name[i] for i in pol], c[pol],color='green',label='Positive')
plt.barh([name[i] for i in otr],c[otr],color='red',label='Negative')
#не смогла исправить, простите за страшную картинку и код....
